In [52]:
import datetime
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
matching = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_drones-with-matching-cfs_matched-by-ID.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
cv_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-flight-manifest.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
cfs = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cfs-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
types = pd.read_csv('../../data/manually-label/labelled-types.csv')

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [44]:
cv_flights = pd.merge(cv_flights,types, on='clean-type',how='left')

In [32]:
cfs = pd.merge(cfs,types, on='clean-type',how='left')

In [60]:
matching[(matching['Description'].astype(str).str.contains('party',case=False))|(matching['Description'].astype(str).str.contains('noise',case=False))][['type','Description']]

,type,Description
21,Stolen Vehicle,Disturbance - Noise
1254,Subject Causing Disturbance,Disturbance - Party
3770,Unknown problem,Disturbance - Noise
5562,Loud Music,Disturbance - Noise
5835,Noise Complaint,Disturbance - Noise
6472,652 Moss St,Disturbance - Party
6894,Subjects Causing Disturbance,Disturbance - Noise
7113,Noise Complaint,Disturbance - Party
7613,Shots Heard,Disturbance - Noise


In [47]:
cv_flights

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,clean-type,match,Predicted Category
0,32.63998083,-117.08271223,1:55pm,55a1fb6dab54e24b8537c7f12a4fe725,2021-07-01,1:55pm,315 4th Ave,NaN,NaN,cvpd,NaN,NaN,Unknown
1,32.61408,-116.975275,4:20pm,15759cef724c96374c1a93419d41dd53,2021-07-01,4:20pm,WOLF CANYON/PEABODY,L054372,DOMESTIC VIOLENCE,cvpd,domestic violence,54372,Violent Crime
2,32.630147,-117.094228,4:09pm,b7235cf661360dc6e3ad6dad7aff335f,2021-07-01,4:09pm,745 H,L054334,UNKNOWN PROBLEM,cvpd,unknown problem,54334,Unknown
3,32.6483145,-117.092483,4:02pm,171d95280b64377eafadaef80590acd0,2021-07-01,4:02pm,500 SEA VALE CT,L054331,TRAFFIC COLLISION,cvpd,traffic collision,54331,Traffic
4,32.6175204,-117.0713635,3:05pm,2dde46cad7d7d0103a9b42dcf9514aec,2021-07-01,3:05pm,330 MOSS,L054364,DOMESTIC VIOLENCE,cvpd,domestic violence,54364,Violent Crime
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9554,32.5989197,-117.0615246,2:14pm,e4fdf51abb9f2d999af03177b0e2595d,2023-09-02,2:14pm,275 Orange Ave.,CVL75348,Domestic Violence,cvpd,domestic violence,75348,Violent Crime
9555,32.6098649,-117.0813317,12:55pm,32c3272b9e976103882e5fe03b020112,2023-09-02,12:55pm,Broadway/ Naples St.,CVL75330,Traffic Hazard,cvpd,traffic hazard,75330,Traffic
9556,32.6495326,-117.0285142,2:04am,5365db931f56c1c04dea07c19813dc7e,2023-09-02,2:04am,400 Milagrosa Circle,L075219,Burglary,cvpd,burglary,75219,Property Crime
9557,32.59246813605942,-117.0126979673828,6:40pm,136cbafe6c19237c10fd27fd99fb75ad,2023-09-02,6:40pm,800 Main St.,CVL75399,Duress Alarm,cvpd,duress alarm,75399,Suspicious Circumstances


In [38]:
df = []
for t in types['Predicted Category'].drop_duplicates():
    number_flights = cv_flights[cv_flights['Predicted Category']==t].shape[0]
    number_calls = cfs[cfs['Predicted Category']==t].shape[0]
    row = {}
    row['num_drones']=number_flights
    row['num_calls']=number_calls
    row['type'] = t
    df.append(row)
    


In [40]:
df = pd.DataFrame(df)

In [41]:
df['percent'] = df['num_drones']/df['num_calls']

In [42]:
df

,num_drones,num_calls,type,percent
0,1383,28290,Unknown,0.048887
1,16,0,Mislabel,inf
2,886,3924,Violent Crime,0.225790
3,270,633,Person with Weapon,0.426540
4,1187,13182,Suspicious Circumstances,0.090047
5,248,7273,Wanted Person/Vehicle,0.034099
6,1710,30435,Disturbance,0.056185
7,1333,10083,Welfare Check,0.132203
8,801,21062,Property Crime,0.038031
9,1222,32168,Traffic,0.037988
